# Song Recommender Program Using Unsupervised Learning through the K-Nearest Neighbors Algorithm

First, we'll want to create a virtual environment in Python. We want to do this because virtual environments essentially give you a way to isolate your dependencies from other Python programs. Essentially, since we'll be installing and using Python packages in this demo, we don't want these packages to interfere with any other Python programs that you may have, so we'll just create our own virtual environment for this. Run the two commands below in the terminal to setup a virtual environment.

-   python3 -m venv myenv

-   source myenv/bin/activate

Then, install pandas and scikit-learn (which are packages that we'll be using) by running the two commands below and wait for it to download.

-   pip install pandas

-   pip install scikit-learn

After that, you may need to select a kernel. Do this by clicking the 'kernel' button in the top-right and setting it to your virtual environment. You may need to install ipykernel to do this, which you should do.


In [52]:
# import pandas
import pandas as pd
 
# scikit-learn imports
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

In general, our steps that we'll take are as follows:

1. Read in the dataset and convert it to a pandas dataframe.

2. Explore the dataset by looking at its dimensions, what columns it has, etc.

3. Clean the dataset by getting rid of any duplicates, missing values, and unneeded columns.

4. Convert categorical data into numerical data that can be used for our algorithm.

5. Scale the data to minimize biases.

6. Create the KNN model and ask the user for an input track.

This dataset was taken from Kaggle at https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset. It contains data for over 100,000 tracks across over 125 different genres. It was last updated 2 years ago, so the program won't be able to accept songs released after then.

In [53]:
# read in the csv file and create a dataframe
tracks = pd.read_csv('dataset.csv')
pd.set_option('display.max_columns', None)

In [72]:
# print out the first 5 rows
tracks.head()


Filtered data has been written to output.csv


In [55]:
# print out dimensions of df (rows, columns)
tracks.shape

(114000, 21)

In [56]:
# print out the columns to see what attributes each track has
tracks.columns

Index(['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name',
       'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'track_genre'],
      dtype='object')

As you may have noticed, the quality of our machine learning model is heavily reliant on our dataset. Therefore, we need to make sure that our dataset is clean in order to produce high-quality results. This includes dealing with missing/NULL values, duplicate data, and irrelevant data in our dataset. 

In [57]:
# print out if we have any duplicates in our dataset. Looks like there's not!
tracks.duplicated()

# see if there's missing data in any of the columns
tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16  liveness          11

In [58]:
# filter the columns into categorical and numerical data

# categorical columns
cat_col = [col for col in tracks.columns if tracks[col].dtype == 'object']
print('Categorical columns :',cat_col)
# numerical columns
num_col = [col for col in tracks.columns if tracks[col].dtype != 'object']
print('Numerical columns :',num_col)



Categorical columns : ['track_id', 'artists', 'album_name', 'track_name', 'track_genre']
Numerical columns : ['Unnamed: 0', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']


In [59]:
# we see that there are some rows with the same track ID. This means that we have duplicate songs!
tracks[cat_col].nunique()

unique_tracks = tracks.drop_duplicates().reset_index(drop=True)
duplicates = unique_tracks['track_id'].duplicated()

print("Rows with duplicates in 'track_id' column:")
(unique_tracks[duplicates])['track_name']

Rows with duplicates in 'track_id' column:


1925                   Song for Rollins
2155                      Snow (Hey Oh)
3000                       Daddy Issues
3002                           Softcore
3003                    Sweater Weather
                      ...              
113572         Jesus We Love You - Live
113605    In The Ordinary - Spontaneous
113617                 King of My Heart
113619                           Simple
113641                           Closer
Name: track_name, Length: 24259, dtype: object

We still have some duplicates because some songs are remixes, so they have different track IDs. Thus, it makes it harder to find these remixes and get rid of them. You could probably utilize regex's to do this, but for the purposes of this workshop, we'll just leave in the remixes. They shouldn't affect the model that much since there are over 100,000 songs in the dataset.

In [60]:
# remove a feature that we aren't interested in
unique_tracks = unique_tracks.drop(['track_id'], axis = 1)

In order for our machine learning algorithm to work, we'll need each categorical variable to be converted into real numbers. We can do this by using label encoding, which assigns each unique genre a number.

In [ ]:
# print the unique genres
unique_genres = unique_tracks["track_genre"].unique()
print(unique_genres)

# create a LabelEncoder() and then use it to fit and transform the 'track_genre' data
label_encoder = LabelEncoder()
unique_tracks['track_genre'] = label_encoder.fit_transform(unique_tracks['track_genre'])

['acoustic' 'afrobeat' 'alt-rock' 'alternative' 'ambient' 'anime'
 'black-metal' 'bluegrass' 'blues' 'brazil' 'breakbeat' 'british'
 'cantopop' 'chicago-house' 'children' 'chill' 'classical' 'club' 'comedy'
 'country' 'dance' 'dancehall' 'death-metal' 'deep-house' 'detroit-techno'
 'disco' 'disney' 'drum-and-bass' 'dub' 'dubstep' 'edm' 'electro'
 'electronic' 'emo' 'folk' 'forro' 'french' 'funk' 'garage' 'german'
 'gospel' 'goth' 'grindcore' 'groove' 'grunge' 'guitar' 'happy'
 'hard-rock' 'hardcore' 'hardstyle' 'heavy-metal' 'hip-hop' 'honky-tonk'
 'house' 'idm' 'indian' 'indie-pop' 'indie' 'industrial' 'iranian'
 'j-dance' 'j-idol' 'j-pop' 'j-rock' 'jazz' 'k-pop' 'kids' 'latin'
 'latino' 'malay' 'mandopop' 'metal' 'metalcore' 'minimal-techno' 'mpb'
 'new-age' 'opera' 'pagode' 'party' 'piano' 'pop-film' 'pop' 'power-pop'
 'progressive-house' 'psych-rock' 'punk-rock' 'punk' 'r-n-b' 'reggae'
 'reggaeton' 'rock-n-roll' 'rock' 'rockabilly' 'romance' 'sad' 'salsa'
 'samba' 'sertanejo' 'show

In [62]:
# show the dataframe with the encoded genre
unique_tracks["track_genre"].unique()
non_unique_rows = unique_tracks[unique_tracks.duplicated(keep=False)]
non_unique_rows.head()

,Unnamed: 0,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre


In [63]:
# let's also get rid of songs with duplicate track names

# TODO: drop rows that have duplicate track names

unique_tracks = unique_tracks.drop_duplicates(subset=['track_name'], keep='first').reset_index(drop=True)
non_unique_rows = unique_tracks[unique_tracks.duplicated(keep=False)]

print(len(non_unique_rows))

0


As before with the 'track_genre' feature, we have another feature, the 'explicit' feature, which represents a boolean that is True if the song is explicit and False if not. Here, we can use one-hot encoding to convert each value to a 0 or a 1 to work with our algorithm.

In [64]:
# TODO: utilize one-hot encoding to convert the 'explicit' feature into a binary format

unique_tracks['explicit'] = unique_tracks['explicit'].astype(int)
print(unique_tracks['explicit'].unique())

[0 1]


In [65]:
def scale_columns():
    '''In this function, we'll be scaling the data by standardizing it, making sure that it's a dataframe,
    printing its head and shape, and then returning the scaled dataframe.'''
    
    # TODO: create a StandardScaler() object, fit the scaler to the numerical data and transform the data, 
    # convert to dataframe
    scaler = StandardScaler()

    # fit the scaler to the data and transform it
    scaled_data = scaler.fit_transform(unique_tracks.drop(columns=['album_name', 'track_name', 'artists']))
    scaled_df = pd.DataFrame(scaled_data, columns=unique_tracks.columns.drop(['album_name', 'track_name', 'artists']))

    scaled_df.head()
    scaled_df.shape
    return scaled_df
    

In [66]:
def setup_knn(scaled_df):
    '''Initializes the K-Nearest Neighbors model and fits it on the given scaled dataframe.'''
    
    # TODO: create the model, letting k = 10 and the distance formula be 'euclidean'. Then, fit it to the data
    knn = NearestNeighbors(n_neighbors=10, metric='euclidean')
    knn.fit(scaled_df)
    return knn

In [67]:
def recommend_songs(track, knn, scaled_data):
    '''Given an input track, checks if the track is in the dataset. If so, it retrieves the row number of its first
    appearance and uses the K-Nearest Neighbors model to find the 10 nearest neighbors. Then, it prints the 10 tracks
    and corresponding artists.'''
    

    if track not in unique_tracks['track_name'].values:
        print("Track not in dataset")
    else:
        # TODO: find the index of the track in the dataframe, then find the indices of the 10 closest tracks
        row_index = unique_tracks[unique_tracks['track_name'] == track].index[0]
        unique_tracks.iloc[row_index]
        
        distances, indices = knn.kneighbors(scaled_data.iloc[row_index].values.reshape(1, -1))

        ret_tracks = []
        # TODO: for each of the 10 closest tracks, append the 'track_name' and 'artists' features to the list
        for i in indices:
            ret_tracks.append((unique_tracks.iloc[i]['track_name'], unique_tracks.iloc[i]['artists']))
        print(ret_tracks)

Now, let's call our functions and ask the user to input a track.

In [68]:
scaled_data = scale_columns()
knn = setup_knn(scaled_data)
input_track = input("Enter a track: ")
recommend_songs(input_track, knn, scaled_data)

[(9070                 Hello
9102     Chasing Pavements
9468     Keep Your Head Up
16707         Good For You
2686           Akuma no Ko
2011      Carnival of Rust
133             Had It All
12804        Can't Pretend
4509                  春はゆく
2414                Heroes
Name: track_name, dtype: object, 9070                       Adele
9102                       Adele
9468                  Ben Howard
16707    Selena Gomez;A$AP Rocky
2686                  Higuchi Ai
2011           Poets of the Fall
133                    Parachute
12804                  Tom Odell
4509                       Aimer
2414                  Zayde Wølf
Name: artists, dtype: object)]


/Users/nathanphan/Desktop/python projects/workshop/myenv/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


After you're done with running the project, feel free to deactivate your virtual environment by running the following command in the terminal:

- deactivate